In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import requests, json, os, datetime, traceback, bs4, webbrowser
import pandas as pd
import numpy  as np
from time import sleep


In [2]:
headers = {
    "Host": "egrul.nalog.ru",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://egrul.nalog.ru/index.html",
    "Content-Type": "application/x-www-form-urlencoded",
    "X-Requested-With": "XMLHttpRequest"
    }

In [155]:
ff = [
#1000000000,
7736207543,
23,
1661056318,
111111111111111,
5752030868,
6732161906,
]
gg = pd.DataFrame(ff, columns=['inn'])
gg.to_excel('gg.xlsx', index=False)
gg = pd.read_excel('gg.xlsx')
gg
#gg['inn'].values.tolist()
#str(np.array(gg['inn']))

,inn
0,7736207543
1,23
2,1661056318
3,111111111111111
4,5752030868
5,6732161906


In [156]:
querys = gg['inn'].values.tolist()
#querys = ['7707510023', 'ООО Яндекс', '1057749528100', '7736207543', '1661056318','5752030868','6732161906']
s = requests.Session()
n = 0

for item in querys:
    query = querys[n]
    r = s.get("https://egrul.nalog.ru/index.html",
        headers=headers
        )

#запроси по ИНН
    req = requests.Request(
        'POST',
        'https://egrul.nalog.ru/',
        data=b'vyp3CaptchaToken=&page=&query='+str(query).encode()+b'&region=&PreventChromeAutocomplete=', 
        headers = headers
        )
    r = s.prepare_request(req)
    r = s.send(r)
    item = json.loads(r.text)
    
# работа с капчей 
    try:
        if item["ERRORS"] != '' and (item["ERRORS"])["captchaSearch"] != '':
            while True:
                r = s.get('https://egrul.nalog.ru/captcha-dialog.html',
                headers = {
                    "Host": "egrul.nalog.ru",
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
                    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
                    "Referer": "https://egrul.nalog.ru/index.html",
                    "Pragma": "no-cache",
                    "Cache-Control": "no-cache"
                })
                b = bs4.BeautifulSoup(r.content.decode(),features="lxml").find('div',class_='field-data').find('img').get('src')
                webbrowser.open('https://egrul.nalog.ru' + b)
                ct = b.split('?a=')[1].split('&')[0]
                captcha = input('Введите капчу: ')

                r = requests.Request(
                'POST',
                'https://egrul.nalog.ru/captcha-proc.json',
                data=b'captcha='+captcha.encode()+b'&captchaToken='+ct.encode(), 
                headers = {
                    "Host": "egrul.nalog.ru",
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
                    "Accept": "application/json, text/javascript, */*; q=0.01",
                    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
                    "Accept-Encoding": "gzip, deflate",
                    "Referer": "https://egrul.nalog.ru/index.html"
                    }
                )

                r = s.prepare_request(req)
                r = s.send(r)
                item = json.loads(r.text)

                try:
                    tr = False
                    if item["ERRORS"] != '':
                        tr = True
                except Exception as e:
                    print(e)
                    pass
                if tr == False: break
                

    except Exception as e:
        pass

#работа капчей

    
    t = json.loads(r.text)['t']

    sleep(0.5)

    r = s.get("https://egrul.nalog.ru/search-result/"+str(t),
        headers = headers
        )

    jsn = json.loads(r.text) #словарь 

    try:
        while True:
            if jsn['status'] != 'wait': break
            sleep(0.2)
    except Exception:
        pass

    try:
        item = (jsn["rows"])[0]  #ключ
                
        t = item['t']

        r = s.get("https://egrul.nalog.ru/vyp-request/"+str(t),  #загрузочная URL
            headers=headers
            )
        sleep(0.5)
        while True:
            r = s.get("https://egrul.nalog.ru/vyp-status/"+str(t),
                headers = headers
                )
            st = json.loads(r.text)['status']
            if st == 'ready': break
            sleep(0.5)

        r = s.get("https://egrul.nalog.ru/vyp-download/"+str(t),
                headers = headers
                )


        f = open(str(query) + '.pdf','wb+')
        f.write(r.content)
        f.close()
        n += 1
#        sleep(5)

    except Exception as e:
        traceback.print_exc()
        pass


Traceback (most recent call last):
  File "<ipython-input-156-86df49ea7627>", line 105, in <module>
    st = json.loads(r.text)['status']
  File "C:\Users\afecn\anaconda3.1\lib\json\__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "C:\Users\afecn\anaconda3.1\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\afecn\anaconda3.1\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 3 column 1 (char 2)


In [48]:
"https://egrul.nalog.ru/vyp-download/"+str(t)

'https://egrul.nalog.ru/vyp-download/21EB7609794A3184E83FDD9890897128582901E5B498BC6D36DA64A9FD1722D22D3CC74FADF2EDE6A2BD1BEE5153B77B7390D4E3BDD4B2570DB7102265B9685D'